# Generation of labeled voltage phasors under both physical attack and FDI attack via SMART-DS dataset and OpenDSS software:

Generate voltage phasors with attack labels via:
  1 SMART-DS, a large open-source dataset of distribution networks with realistic (active/reactive power) load profile in different voltage levels (see details below)
  2 OpenDSS, a widely-used open-source software for 3-phase unbalanced distribution network power flow analysis

Two types of attacks are considered:
  1 Physical attacks, which manipulate VV/VW control curve of inverters
  2 Stealthy false data injection (FDI) attacks, which manipulate measurement to bypass traditional bad data detection

# SMART-DS dataset details:

In SMART-DS dataset, 3-year load profiles are included: 2016, 2017, 2018. In addition, more than 3 voltage levels are considered:
  1 230 kV: subtransmission level
  2 69 kV: substation level
  3 4 kV ~ 25 kV: feeder levels 

The data structure of SMART-DS dataset is organized as follows:
SMART-DS
└─── <GIS>
└─── <PLACEMENTS>
└─── <YEARS>
    └─── <DATASETS>
         └──── full_dataset_analysis
         └─── <SUB-REGIONS>
              └──── profiles
              └──── cyme_profiles
              └──── load_data
              └──── solar_data
              └──── load_curves
              └──── <SCENARIOS>
                     └──── metrics.csv
                     └──── cyme
                     └──── opendss
                     │     └──── analysis
                     │     └──── <SUBSTATIONS>
                     │           └──── analysis
                     │           └──── <FEEDERS>
                     │                 └──── analysis
                     └──── opendss_no_loadshapes
                           └──── <SUBSTATIONS>
                                 └──── <FEEDERS>

# How to start:
1. Unzip the .zip file
2. Create new conda environment with python=3.11
3. In the new conda env: 
    pip install .
    pip install torch==2.0.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
    pip install cplxmodule
    pip3 install scikit-learn
    pip install numpy==1.24.4
    pip install pyarrow
4. Restart and run this .ipynb file

In [ ]:
# 0 import
from data_loader import RolloutStorage, data_loader_FDIPhy
from graph_loader import single2batch_phy
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from cplxmodule import cplx
from cplxmodule.nn.modules import CplxConv1d
from cplxmodule.nn.modules import CplxLinear
from model.layers import ChebGraphConv
from model.utility import calc_gso, calc_chebynet_gso, cnv_sparse_mat_to_coo_tensor
from cplxmodule.nn import CplxToCplx
from torch.autograd import Variable
import opendssdirect as dss
import os
import pandas as pd
import matplotlib.pyplot as plt
import scipy.sparse as sps
from scipy.sparse import csc_matrix, diags
import random
import measurements
from scipy.sparse.linalg import svds
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#random.seed(100) 

# 0 check GPU
# Check if CUDA is available
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    # Get the current device
    current_device = torch.cuda.current_device()
    print("Current CUDA Device Index:", current_device)
    # Get device name
    print("Current CUDA Device Name:", torch.cuda.get_device_name(current_device))
    # Check CUDA version
    print("CUDA Version:", torch.version.cuda)
# check version of numpy and torch
print("NumPy version:", np.__version__)
print("PyTorch version:", torch.__version__)
# check if NumPy is available
try:
    arr = np.array([1, 2, 3], dtype=np.float32)
    print("NumPy array:", arr)
except Exception as e:
    print("NumPy error:", e)
# check if Numpy is compatible with PyTorch
try:
    tensor = torch.from_numpy(arr)
    print("Torch tensor:", tensor)
except Exception as e:
    print("Torch error:", e)

In [ ]:
# 1 select a 3-phase unbalanced distribution system from the SMART-DS dataset
# according to the specific data structure of SMART-DS dataset
current_dir = os.getcwd()
#print("current work folder:", current_dir)
area = 'P4U'
sce = 'scenarios'
timeseries = 'base_timeseries'
loadshape = 'opendss_no_loadshapes'
substation = 'p4uhs0_4'
feeder = 'p4uhs0_4--p4udt4'
Master_dss = os.path.join(area, sce, timeseries, loadshape, substation, feeder, 'Master.dss')

In [ ]:
# 2 initialize OpenDSS
dss.Basic.ClearAll() # Initialize OpenDSS
result = dss.run_command("Redirect " + Master_dss)

NumBus = dss.Circuit.NumBuses()
NumNode = dss.Circuit.NumNodes()
print(NumBus)
print(NumNode)

In [ ]:
# 3 extract Y matrix of the selected SMART-DS example using OpenDSS
Y_NodeOrder = dss.Circuit.YNodeOrder()
Y_sparse = sps.csc_matrix(dss.YMatrix.getYsparse())
# Y_sparse type is scipy.sparse.csc_matrix
# e.g., Y_sparse = csc_matrix((data, (row, col)), shape=(n, n))
# Extract diagonal matrix from Ybus
Dmat = diags(Y_sparse.diagonal(), format='csc')
# Compute the inverse square root of the diagonal matrix
Dmat_inv_sqrt = diags(1 / np.sqrt(Dmat.diagonal()), format='csc')
# Normalize the admittance matrix
Y_norm = Dmat_inv_sqrt @ Y_sparse @ Dmat_inv_sqrt
# Filter small values in Y_norm
Y_norm = Y_norm.toarray()  # Convert to dense matrix for element-wise operations
real_small = np.abs(Y_norm.real) < 1e-8
imag_small = np.abs(Y_norm.imag) < 1e-8
Y_norm.real[real_small] = 0
Y_norm.imag[imag_small] = 0
# Convert back to sparse format
Y_norm_sparse = csc_matrix(Y_norm)
Y_dense = Y_norm_sparse.toarray()
from scipy.linalg import svd

# SVD of normalized Y matrix
U, S, Vh = svd(Y_dense, full_matrices=False)

# Select top-k left singular vectors
k = 80
U_k = U[:, :k]

print(Y_NodeOrder)
print(Y_norm_sparse)

In [ ]:
energy = np.cumsum(S**2) / np.sum(S**2)
print(energy)
plt.plot(S, marker='o') 
plt.axvline(x=75, color='r', linestyle='--', label='k=80')
plt.xlabel("Singular Vector Index")
plt.ylabel("Singular Value")
plt.title("Singular Values of Normalized Y Matrix")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#No need for this block.. it just prints out system specs for my reference
import opendssdirect as dss

# Print system summary
print("System Overview:")
print(f"  Number of Buses: {dss.Circuit.NumBuses()}")
print(f"  Number of Nodes: {dss.Circuit.NumNodes()}")
print(f"  Number of Loads: {dss.Loads.Count()}")
print(f"  Number of Lines: {dss.Lines.Count()}")
print(f"  Number of Transformers: {dss.Transformers.Count()}")

# Bus info
print("\nBus Info:")
for bus in dss.Circuit.AllBusNames():
    dss.Circuit.SetActiveBus(bus)
    kv_base = dss.Bus.kVBase()
    phases = dss.Bus.Nodes()
    print(f"  Bus: {bus}, kV base: {kv_base:.3f}, Phases: {phases}")

# Load info
print("\nLoad Info:")
dss.Loads.First()
while True:
    name = dss.Loads.Name()
    kw = dss.Loads.kW()
    kvar = dss.Loads.kvar()
    kv = dss.Loads.kV()
    phases = dss.Loads.Phases()
    print(f"  Load: {name}, kW: {kw}, kvar: {kvar}, kV: {kv}, Phases: {phases}")
    if not dss.Loads.Next() > 0:
        break

import cmath

print("\nBus Phase Voltage Base Mapping:")

for bus in dss.Circuit.AllBusNames():
    dss.Circuit.SetActiveBus(bus)
    kv_base = dss.Bus.kVBase()
    phases = dss.Bus.Nodes()
    for ph in phases:
        node = f"{bus}.{ph}"
        print(f"  Node: {node}, kV base LN: {kv_base:.4f}")

def print_ycurrents():
    currents = dss.Circuit.YCurrents()
    N = len(currents) // 2
    print("\nOpenDSS YCurrents (Net Injections):")
    for i in range(N):
        re, im = currents[2*i], currents[2*i+1]
        current = complex(re, im)
        print(f"  Node {i+1}: I = {current:.4f} A")
print_ycurrents()
def print_element_currents():
    print("\nElement Currents (Phase-resolved):")
    for elem in dss.Circuit.AllElementNames():
        dss.Circuit.SetActiveElement(elem)
        name = dss.CktElement.Name()
        buses = dss.CktElement.BusNames()
        currents = dss.CktElement.Currents()
        print(f"\nElement: {name}")
        for i, bus in enumerate(buses):
            print(f"  Terminal {i+1}: {bus}")
        for i in range(0, len(currents), 2):
            current = complex(currents[i], currents[i+1])
            print(f"    I = {current:.4f} A")
print_element_currents()
print("\nElement Conductor Count Check:")
for elem in dss.Circuit.AllElementNames():
    dss.Circuit.SetActiveElement(elem)
    print(f"{elem}: {dss.CktElement.NumConductors()} conductors, {dss.CktElement.NumTerminals()} terminals")
def print_actual_voltages():
    print("\nActual Voltages (V):")
    for bus in dss.Circuit.AllBusNames():
        dss.Circuit.SetActiveBus(bus)
        voltages = dss.Bus.VMagAngle()
        phases = dss.Bus.Nodes()
        for i, ph in enumerate(phases):
            vmag = voltages[2 * i]
            vang = voltages[2 * i + 1]
            print(f"{bus}.{ph}: {vmag:.2f} V ∠ {vang:.2f}°")
print_actual_voltages()
# Loop over all buses and print per-unit voltages (magnitude and angle)
for bus in dss.Circuit.AllBusNames():
    dss.Circuit.SetActiveBus(bus)
    pu_voltages = dss.Bus.PuVoltage()  # returns [Vr1, Vi1, Vr2, Vi2, ...]
    num_phases = int(len(pu_voltages) / 2)

    print(f"Bus: {bus}")
    for i in range(num_phases):
        vr = pu_voltages[2 * i]
        vi = pu_voltages[2 * i + 1]
        voltage = complex(vr, vi)
        mag = abs(voltage)
        angle = cmath.phase(voltage) * 180 / np.pi
        print(f"  Phase {i+1}: |V| = {mag:.4f} pu, ∠ = {angle:.2f}°")


In [ ]:
# 4 extract the PQ load profile for the selected SMART-DS example
# according to the specific data structure of SMART-DS dataset
Loads_dss = os.path.join(area, sce, timeseries, loadshape, substation, feeder, 'Loads.dss')
load_data_folder = os.path.join(area, 'load_data')
# Initialize an empty dictionary to store load mappings
load_profile_map = {}
# Open the Loads.dss file for reading
with open(Loads_dss) as f_load:
    # Iterate through each line in the file
    for row in f_load.readlines():
        # Split the line into tokens (words) using whitespace as a delimiter
        sp = row.split()
        # Initialize temporary variables for this line
        name = None  # Load name
        profile = None  # Time series profile file name
        multiplier = 1  # Default multiplier for loads
        # Iterate through each token in the line
        for token in sp:
            # If the token represents a load definition, extract the load name
            if token.startswith('Load.'):
                # Extract the load name after the 'Load.' prefix
                name = token.split('.')[1]
                # Check if the load name ends with '_1' or '_2'
                # These indicate center-tap loads, so we set the multiplier to 0.5
                if name.endswith('_1') or name.endswith('_2'):
                    multiplier = 0.5  # Center-tap loads
            # If the token contains the yearly time series profile information
            # e.g., !yearly=com_kw_16145_pu
            if token.startswith('!yearly'):
                # Extract the profile information after the '=' sign
                profile_raw = token.split('=')[1]
                # Special case: if the profile indicates a mesh load
                if 'mesh' in profile_raw:
                    profile = 'mesh'
                else:
                    # Parse the profile name into components and construct the .parquet file name
                    profile_sp = profile_raw.split('_')
                    profile = profile_sp[0] + '_' + profile_sp[2] + '.parquet'  # e.g., com_16145.parquet
            # If a profile has been identified, store the load information in the dictionary
            if profile is not None:
                # Add the load name as the key and a tuple (profile, multiplier) as the value
                load_profile_map[name] = (profile, multiplier)
# The resulting `load_profile_map` contains mappings of load names to their respective
# time series profile files and multipliers (for center-tap loads).

sample_profile = next(iter(load_profile_map.values()))[0]
parquet_path_sample = os.path.join(load_data_folder, sample_profile)
total_timepoints = len(pd.read_parquet(parquet_path_sample))
sampling_rate = 20 

In [ ]:
# --- Step 5: Initialize the PV inverter ---

# Select 30 PV inverters (feeder names)
pv_feeder = list(load_profile_map.keys())[:30]  
num_pv = len(pv_feeder)

# Function to determine whether a PV inverter is connected to phase 2
def is_phase2(feeder_name):
    try:
        dss.Loads.Name(feeder_name)  # Activate the load
        nodes = dss.CktElement.NodeOrder()  # Phase numbers it's connected to
        return 2 in nodes
    except Exception as e:
        print(f"Warning: Could not resolve feeder {feeder_name} → {e}")
        return False

# Print connected phases for verification (optional debug)
print("\n--- PV Feeder Phase Connection ---")
for feeder in pv_feeder:
    dss.Loads.Name(feeder)
    phases = dss.CktElement.NodeOrder()
    print(f"{feeder} → Connected Phases: {phases}")

# Determine which PV inverters are connected to phase 2
pv_phase2_flags = [is_phase2(feeder) for feeder in pv_feeder]


# Initialize VV/VW control parameters
# Shape: (num_pv, 3 phases, 5 control points)
eta_base = np.array([0.95, 0.97, 1.03, 1.05, 1.10])
eta = np.tile(eta_base, (num_pv, 3, 1))  # Same for all phases initially

# Set PV inverter rated power
sv_power = 2.96  # kVA

# Generate random PV active power profiles
np.random.seed(100)
pv_power = np.random.uniform(1.0, 1.2, size=(total_timepoints, num_pv))

# Reshape feeder names to match OpenDSS node format (e.g., load_p4ulv5_2 → p4ulv5.2)
pv_feeder_reshape = [feeder.replace("load_", "").replace("_", ".") for feeder in pv_feeder]


In [ ]:
def inject_false2(eta, pv_phase2_flags, attack_fraction=0.8, new_curve=None):
    eta_comm = eta.copy()
    attack_label = np.zeros(eta.shape[0], dtype=int)

    # Get all PV indices connected to phase 2
    phase2_indices = [i for i, is_p2 in enumerate(pv_phase2_flags) if is_p2]

    # Compute how many to randomly attack (80% of them)
    num_phase2 = len(phase2_indices)
    num_to_attack = int(np.floor(num_phase2 * attack_fraction))

    # Randomly select PVs to attack
    if num_to_attack > 0:
        selected = np.random.choice(phase2_indices, num_to_attack, replace=False)
    else:
        selected = []

    # Default new control curve if not provided
    if new_curve is None:
        new_curve = np.array([0.98, 0.99, 1.01, 1.02, 1.10])

    # Apply the new control curve to the 2nd phase (index 1)
    for idx in selected:
        eta_comm[idx, 1, :] = new_curve
        attack_label[idx] = 1

    return eta_comm, attack_label

def VW_func(Vm_pv_lp_t1, pv_power, eta, phase_idx):
    eta_4 = eta[:, phase_idx, 3]
    eta_5 = eta[:, phase_idx, 4]
    Pw_inj = np.zeros_like(Vm_pv_lp_t1)
    for i in range(len(Vm_pv_lp_t1)):
        if Vm_pv_lp_t1[i] <= eta_4[i]:
            Pw_inj[i] = pv_power[i]
        elif Vm_pv_lp_t1[i] <= eta_5[i]:
            Pw_inj[i] = (eta_5[i] - Vm_pv_lp_t1[i]) / (eta_5[i] - eta_4[i]) * pv_power[i]
        else:
            Pw_inj[i] = 0
    return Pw_inj

def VV_func(Vm_pv_lp_t1, qv_power, eta_phase):

    eta_1 = eta_phase[:, 0]
    eta_2 = eta_phase[:, 1]
    eta_3 = eta_phase[:, 2]
    eta_4 = eta_phase[:, 3]

    Qv_inj = np.zeros_like(Vm_pv_lp_t1)
    for i in range(len(Vm_pv_lp_t1)):
        V = Vm_pv_lp_t1[i]
        qmax = qv_power[i]

        if V <= eta_1[i]:
            Qv_inj[i] = qmax
        elif V <= eta_2[i]:
            Qv_inj[i] = (eta_2[i] - V) / (eta_2[i] - eta_1[i]) * qmax
        elif V <= eta_3[i]:
            Qv_inj[i] = 0
        elif V <= eta_4[i]:
            Qv_inj[i] = -(V - eta_3[i]) / (eta_4[i] - eta_3[i]) * qmax
        else:
            Qv_inj[i] = -qmax

    return Qv_inj


def VV_VW_func(Vm_pv, eta, pv_power, sv_power, Vm_pv_lp_t0, pinj_pv_t0, qinj_pv_t0, target_phase=2):

    tau_c = 0.98
    tau_o = 0.98
    Vm_pv_lp_t1 = Vm_pv_lp_t0 + tau_c * (Vm_pv - Vm_pv_lp_t0)

    # VW
    eta_4 = eta[:, target_phase - 1, 3]
    eta_5 = eta[:, target_phase - 1, 4]
    Pw_inj = np.zeros_like(Vm_pv_lp_t1)
    for i in range(len(Vm_pv_lp_t1)):
        if Vm_pv_lp_t1[i] <= eta_4[i]:
            Pw_inj[i] = pv_power[i]
        elif Vm_pv_lp_t1[i] <= eta_5[i]:
            Pw_inj[i] = (eta_5[i] - Vm_pv_lp_t1[i]) / (eta_5[i] - eta_4[i]) * pv_power[i]
        else:
            Pw_inj[i] = 0

    # VV
    qv_power = np.sqrt(np.maximum(sv_power**2 - Pw_inj**2, 0))
    Qv_inj = VV_func(Vm_pv_lp_t1, qv_power, eta[:, target_phase - 1, :])

    # Dynamic filtering
    pinj_pv_t1 = pinj_pv_t0 + tau_o * (Pw_inj - pinj_pv_t0)
    qinj_pv_t1 = qinj_pv_t0 + tau_o * (Qv_inj - qinj_pv_t0)
    PV_inj = pinj_pv_t1 + 1j * qinj_pv_t1

    return PV_inj, Vm_pv_lp_t1, pinj_pv_t1, qinj_pv_t1

def modify_mpc(PV_inj, pv_feeder, active_power_dict, reactive_power_dict):
    injection_map = {}
    for i, feeder in enumerate(pv_feeder):
        injection_map[feeder] = (np.real(PV_inj[i]), np.imag(PV_inj[i]))
    dss.Loads.First()
    while True:
        name = dss.Loads.Name()
        base_kw = active_power_dict.get(name, 0)
        base_kvar = reactive_power_dict.get(name, 0)
        if name in injection_map:
            inj_kw, inj_kvar = injection_map[name]
            new_kw = base_kw + inj_kw
            new_kvar = base_kvar + inj_kvar
        else:
            new_kw = base_kw
            new_kvar = base_kvar
        dss.Loads.kW(new_kw)
        dss.Loads.kvar(new_kvar)
        if not dss.Loads.Next() > 0:
            break

def get_all_node_voltageReIm_map_afterPF():
    dss_map = {}
    bus_names = dss.Circuit.AllBusNames()
    for bus_name in bus_names:
        dss.Circuit.SetActiveBus(bus_name)
        dss_pus = dss.Bus.PuVoltage()
        num_phases = int(len(dss_pus) / 2)
        for i in range(num_phases):
            phase_name = f"{bus_name}.{i + 1}"
            real_part = dss_pus[2 * i]
            imag_part = dss_pus[2 * i + 1]
            dss_map[phase_name] = (real_part, imag_part)
    return dss_map

def get_pv_voltageMag(pv_feeder_reshape):
    dss_map = get_all_node_voltageReIm_map_afterPF()
    print("Example dss_map keys:", list(dss_map.keys())[:10])
    num_pv = len(pv_feeder_reshape)
    Vm_pvMag = np.zeros(num_pv)
    for idx, feeder_reshaped in enumerate(pv_feeder_reshape):
        if feeder_reshaped in dss_map:
            real_part, imag_part = dss_map[feeder_reshaped]
            Vm_pvMag[idx] = abs(complex(real_part, imag_part))
    return Vm_pvMag


def optimal_placement_greedy(U_k, M, M_s=None):
    N = U_k.shape[0]
    if M_s is None:
        M_s = []

    M_tilde = M - len(M_s)
    j_tilde = list(set(range(N)) - set(M_s))

    for _ in range(M_tilde):
        sigma_min_list = []
        for j in j_tilde:
            A = U_k[M_s + [j], :]
            try:
                s = svds(A, k=1, which='SM', return_singular_vectors=False)
                sigma_min_list.append(s[0])
            except:
                sigma_min_list.append(0)

        best_index = np.argmax(sigma_min_list)
        selected_j = j_tilde[best_index]
        M_s.append(selected_j)
        j_tilde.remove(selected_j)

    return M_s


In [ ]:

max_sensor = 120

# Define known PV sensors (30 in total)
pv_sensors_lower = [
    'p4ulv5.1', 'p4ulv5.2', 'p4ulv6.1', 'p4ulv6.2', 'p4ulv10.1', 'p4ulv10.2',
    'p4ulv17.1', 'p4ulv17.2', 'p4ulv18.1', 'p4ulv18.2', 'p4ulv20', 'p4ulv21.1', 'p4ulv21.2',
    'p4ulv22.1', 'p4ulv22.2', 'p4ulv23.1', 'p4ulv23.2', 'p4ulv24.1', 'p4ulv24.2',
    'p4ulv25.1', 'p4ulv25.2', 'p4ulv26.1', 'p4ulv26.2', 'p4ulv27', 'p4ulv28',
    'p4ulv30.1', 'p4ulv30.2', 'p4ulv33', 'p4ulv34.1', 'p4ulv34.2'
]

# Match to canonical node names
fixed_sensors = []
for fs in pv_sensors_lower:
    for sensor in Y_NodeOrder:
        if sensor.lower() == fs:
            fixed_sensors.append(sensor)
            break

# Define initial sensors
initial_sensor_names = fixed_sensors

# Convert to indices
initial_sensor_indices = [Y_NodeOrder.index(name) for name in initial_sensor_names]

# Run greedy optimization
optimal_indices = optimal_placement_greedy(U_k, max_sensor, M_s=initial_sensor_indices.copy())

# Map selected indices to node names
all_optimal_sensors = [Y_NodeOrder[i] for i in optimal_indices]

# For now, treat all as PMUs
S_muPMU = all_optimal_sensors.copy()

# Keep power meters empty, but structure remains
S_PowerMeter = []

# For indexing
sensor_location_indices = [Y_NodeOrder.index(s) for s in S_muPMU]


print("mu-PMU sensor count:", len(S_muPMU))
print("Power meter sensor count:", len(S_PowerMeter))



In [ ]:
# 7 If needed, then generate data for GCN train/test, including:
#   1) voltage phasors at each node at each timepoint
#   2) attack labels at PV inverter at each timepoint

# Reminder: 7 hours to run power flow for this 156-node system at all 35040 timepoints

FlagGen = 0
if FlagGen == 1:
    dss.run_command("Solve")
    ori_Vm_Mag = get_pv_voltageMag(pv_feeder_reshape)
    voltage_map = get_all_node_voltageReIm_map_afterPF()
    node_keys = list(voltage_map.keys())
    num_node = len(dss.Circuit.YNodeOrder())
    # Vall_ReIm_store (total_timepoints, num_node, sampling_rate)
    Vall_ReIm_store = np.empty((total_timepoints, num_node, sampling_rate), dtype=complex)
    # Vpv_mag_store is a 2D array with shape (num_pv, sampling_rate)
    Vpv_mag_store = np.empty((num_pv, sampling_rate), dtype=float)
    # Ppv_store and Qpv_store: 2D arrays (num_pv, sampling_rate)
    Ppv_store = np.empty((num_pv, sampling_rate), dtype=float)
    Qpv_store = np.empty((num_pv, sampling_rate), dtype=float)
    baseline_eta = eta.copy()  # save the original control curve before any attack
    attack_label_store = np.zeros((total_timepoints, num_pv), dtype=int)
    for time_idx in range(total_timepoints):
        print('time index:', time_idx)
        pinj_pv_t0 = np.zeros(num_pv)
        qinj_pv_t0 = np.zeros(num_pv)
        active_power_dict = {}
        reactive_power_dict = {}
        for load_name, (profile, multiplier) in load_profile_map.items():
            parquet_path = os.path.join(load_data_folder, profile)
            parquet_data = pd.read_parquet(parquet_path)
            kw = parquet_data.loc[time_idx, 'total_site_electricity_kw'] * multiplier
            kvar = parquet_data.loc[time_idx, 'total_site_electricity_kvar'] * multiplier
            active_power_dict[load_name] = kw
            reactive_power_dict[load_name] = kvar
        eta_comm, _ = inject_false2(eta, pv_phase2_flags, attack_fraction=0.8)
        time_varying_label = np.zeros(num_pv, dtype=int)
        for i in range(num_pv):
            if pv_phase2_flags[i]:
                if np.any(eta_comm[i, 1, :] != baseline_eta[i, 1, :]):
                    time_varying_label[i] = 1
        attack_label_store[time_idx, :] = time_varying_label
        attacked_pvs = [pv_feeder_reshape[i] for i in range(num_pv) if time_varying_label[i] == 1]
        print("Attacked phase-2 PVs:", attacked_pvs)
        Vm_pv_lp_t0 = ori_Vm_Mag
        Vm_pv_Mag = ori_Vm_Mag
        current_pv_power = pv_power[time_idx, :]
        for sample in range(sampling_rate):
            PV_inj, Vm_pv_lp_t1, pinj_pv_t1, qinj_pv_t1 = VV_VW_func(Vm_pv_Mag, eta_comm, current_pv_power, sv_power,
                                                                        Vm_pv_lp_t0, pinj_pv_t0, qinj_pv_t0)
            Vm_pv_lp_t0 = Vm_pv_lp_t1
            pinj_pv_t0 = pinj_pv_t1
            qinj_pv_t0 = qinj_pv_t1
            modify_mpc(PV_inj, pv_feeder, active_power_dict, reactive_power_dict)
            dss.run_command("Solve")
            Vm_pv_Mag = get_pv_voltageMag(pv_feeder_reshape)
            Vpv_mag_store[:, sample] = Vm_pv_Mag
            Ppv_store[:, sample] = np.real(PV_inj)
            Qpv_store[:, sample] = np.imag(PV_inj)
 
    # save the generated data
    np.save('Vall_ReIm_New.npy', Vall_ReIm_store)
    np.save('attack_label_New.npy', attack_label_store)
    # load and check the saved data
    VoltagePhasor = np.load('Vall_ReIm_New.npy')
    PhysicalAttackLabel = np.load('attack_label_New.npy')
    num_time, num_node, num_sample = VoltagePhasor.shape
    print(num_time, num_node, num_sample)
    num_time1, num_pvInverter = PhysicalAttackLabel.shape
    print(num_time1, num_pvInverter)
    assert VoltagePhasor.shape[0] == PhysicalAttackLabel.shape[0], "Mismatch in timepoints"


In [ ]:
# 8 load voltage phasors (with physical attack label) (before state estimation) for GCN
# the two .npy files are also included in "PhysicalAttackDetection20250327.zip"
# TODO: replace with Vall_ReIm_YourName.npy if data are re-generated
file_path1 = os.path.join(current_dir, 'Vall_ReIm_New.npy')
# TODO: replace with attack_label_YourName.npy if data are re-generated
file_path2 = os.path.join(current_dir, 'attack_label_New.npy')
data_obtain = data_loader_FDIPhy(file_path1, file_path2)

In [ ]:
FlagFDI = 1
voltage_scale_factor = 0.005

# Initialize data loader
data_obtain = data_loader_FDIPhy(file_path1, file_path2, Y_NodeOrder=Y_NodeOrder)

# Apply state estimation (with or without FDI attack)
if FlagFDI == 0:
    print("Running with physical attacks only...")
    # Just load physical attack data
    save_prefix = "Physical_Only"
else:
    print("Running with stealthy FDI + voltage noise...")
    data_obtain.state_est_withFDI(
        Y_norm_sparse,
        sensor_location_indices,
        voltage_scale_factor=voltage_scale_factor
    )
    save_prefix = "FDI_Physical_WithVoltageNoise" if voltage_scale_factor else "FDI_Physical"

# === Sanity checks ===
_, Dimbus, _ = data_obtain.data_recover.shape
_, Dimlabel = data_obtain.label_truth.shape
print("Data recover shape:", Dimbus)
print("Label shape:", Dimlabel)

assert Dimbus == len(Y_NodeOrder), "Mismatch between bus count and Y_NodeOrder"
expected_labels = len(sensor_location_indices) + 30
assert Dimlabel == expected_labels, f"Expected {expected_labels} labels, got {Dimlabel}"

# === Save final processed data ===
np.save(f'Vphasor_{save_prefix}.npy', data_obtain.data_recover)
np.save(f'AttackLabel_{save_prefix}.npy', data_obtain.label_truth)

# Save metadata (used later in training/testing)
metadata = {
    "total_timepoints": total_timepoints,
    "sampling_rate": sampling_rate,
    "pv_feeders": pv_feeder,
    "sensor_nodes": all_optimal_sensors,
    "Y_node_order": Y_NodeOrder,
}
np.save("metadata_run_config.npy", metadata)

# Print key stats
num_time, num_node, num_sample = data_obtain.data_input_beforeSE.shape
print(f"Total timepoints: {num_time}")
print(f"Samples per timepoint: {num_sample}")
